## First Attempt: [click here](http://localhost:8888/notebooks/OneDrive%20-%20Agilent%20Technologies/Documents/_staging/Data-Developer-Droolkit/AGILENT/2020_01_10/CofAs_to_move_FIRST_ATTEMPT.ipynb)

---

Can you please move ONLY the files from this folder: 

`G:\C of A's\#CofA with background for web`

(which match EXACTLY to the file name in the attached excel sheet into the folder):

`G:\C of A's\#CofA with background for web\Missing CofAs`

We will be uploading these first to the web and thus web team prefers these to be separated.

---

In [1]:
import os, sys, time
from time import sleep
from pathlib import Path
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import fnmatch, shutil

In [2]:
def main(): # {
    # RE-INSTANTIATE GLOBALS
    global in_file, in_directory, out_directory, outbound_directory, time_start
    # TRY THE FOLLOWING
    try: # {
        print("in_file == " + str(in_file))
        print("and its type:\t" + str(type(in_file)))
        # IMPORT IN AS DATAFRAME
        filenames_df = pd.read_excel(in_file, dtype=np.str)
        filenames_df.to_csv("filenames.csv", index=False)
        # GET DIRECTORY LISTING AS SERIES / DATAFRAME
        dir_list = os.listdir(in_directory)
        # CRETE SERIES OFF LIST
        d1 = pd.Series(data=dir_list)
        # CREATE NEW EMPTY DATAFRAME WITH LIST FROM EXCEL AS INDEX
        corrections_df = pd.DataFrame(data=None, dtype=np.str)
        # ATTACH COLUMNS
        # [2020-01-10]\\correction_df["CofA PDF file name"] = filenames_df["CofA PDF file name"]
        # [2020-01-10]\\correction_df["Directory File"] = d1
        corrections_df["CofA PDF file name"] = d1
        corrections_df.to_csv("corrections.csv", index=False)
        # PRINT OUT BOTH DATAFRAMES (so far...)
        print("number of rows/cols:\t" + str(filenames_df.shape) + " / " + str(len(filenames_df)))
        print("directory == " + str(in_directory))
        print("and its size:\t" + str(len(dir_list)))
        print("number of rows/cols:\t" + str(corrections_df.shape) + " / " + str(len(corrections_df)))
        print("==================================================")
        """
        GO THRU EACH DATAFRAME ROW BY ROW AND CHANGE ALL LETTERS TO LOWERCASE?
        """
        # SET DIFFERENCE OF BOTH DATAFRAMES
        set_diff_df = pd.concat([corrections_df, filenames_df, corrections_df]
                               ).drop_duplicates(keep=False)
        set_diff_df.to_csv("set_diff_df.csv", index=False)
        # GET INTERSECTION OF BOTH DATAFRAMES
        intersected_df = pd.merge(filenames_df, corrections_df, how='inner')
        intersected_df.to_csv("intersected_df.csv", index=False)
        # CREATE NEW EMPTY DATAFRAME WITH LIST OF WHAT TO MOVE
        move_df = pd.DataFrame(data=None, dtype=np.str)
        # SET FIRST COLUMN AS INERSECTED VALUES
        move_df["CofA PDF file name"] = intersected_df["CofA PDF file name"]
        # ATTACH COLUMNS (with directory names...)
        move_df["in_directory"] = str(Path(in_directory))
        move_df["out_directory"] = str(Path(outbound_directory))
        # EXPORT FOR TESTING REASON?
        move_df.to_csv("move_df_1.csv", index=False)
        ##############################################################################
        """
        ITER TUPLES THE DATAFRAME AND CREATE PATHS FOR EACH ROW
        """
        # CREATE LIST TO HOLD FILENAMES WE ARE MOVING
        file_list = []
        for row in move_df.itertuples(index=False, name="CofA"): # {
            # create file path variables
            src_path = os.path.join(str(row[1]), str(row[0]))
            # [2020-01-10]\\print("FILE-PATH:\n" + str(src_path), end='\n\n')
            dst_path = os.path.join(str(row[2]), str(row[0]))
            # [2020-01-10]\\print("DEST-PATH:\n" + str(dst_path), end='\n\n')
            # APPEND filename to list
            file_list.append(str(row[0]))
            ########################
            # ATTEMPT TO MOVE FILE #
            ########################
            print("Moving File...")
            try: # {
                print("From...")
                print(src_path)
                # move file
                shutil.move(src=src_path, dst=dst_path)
            # }
            except: # {
                print("FAILED MOVING!")
            # }
            else: # {
                print("To...")
                print(dst_path)
            # }
        # }
        #############################################################################
        # PRINT OUT LENGTHS
        print("NUMBER TO MOVE:\n\t" + str(len(move_df)))
        num_to_stay = int(len(dir_list) - len(move_df))
        print("NUMBER TO STAY:\n\t" + str(num_to_stay))
        total_num = int(len(dir_list) + len(corrections_df))
        print("OUT OF # TOTAL:\n\t" + str(len(dir_list)))
        print("==============================")
        print(" so after operation... " + str(total_num) + " ?? ")
        print("==============================")
        file_list_df = pd.DataFrame(data=file_list, dtype=np.str)
        # EXPORT .CSV FILE FOR DILUKA
        file_list_df.to_csv("CofAs-moved-" + str(pd.Timestamp.now())[:10] + ".csv", index=False)
    # }
    except: # {
        errorMessage = str(sys.exc_info()[0]) + "\n"
        errorMessage = errorMessage + str(sys.exc_info()[1]) + "\n"
        errorMessage = errorMessage + str(sys.exc_info()[2]) + "\n"
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        typeE = str("TYPE : " + str(exc_type))
        fileE = str("FILE : " + str(fname))
        lineE = str("LINE : " + str(exc_tb.tb_lineno))
        messageE = str("MESG : " + "\n" + str(errorMessage))
        print("\n" + typeE +
              "\n" + fileE +
              "\n" + lineE +
              "\n" + messageE)
    # }
    else: # {
        print("Operation Completed Successfully...")
    # }
    finally: # {
        # GET TIMESTAMP FOR TIME IT ENDED
        time_end = pd.Timestamp.now()
        # DETERMINE RUN-TIME
        run_time = time_end - time_start
        print("RUN-TIME:\n\t" + str(run_time))
    # }
# }

In [3]:
# MAIN BOILERPLATE
if __name__ == "__main__": # {
    ################################
    # INSTANTIATE GLOBAL VARIABELS #
    ################################
    in_file = Path("C:/data/inbound/CofAs to move .xlsx")
    in_directory = Path("G:/C of A's/#CofA with background for web")
    out_directory = Path("C:/data/outbound/2020-01-10/")
    outbound_directory = Path("G:/C of A's/#CofA with background for web/Missing CofAs")
    time_start = pd.Timestamp.now()
    ######################
    # CALL MAIN FUNCTION #
    ######################
    main()
# }

in_file == C:\data\inbound\CofAs to move .xlsx
and its type:	<class 'pathlib.WindowsPath'>
number of rows/cols:	(11335, 1) / 11335
directory == G:\C of A's\#CofA with background for web
and its size:	58194
number of rows/cols:	(58194, 1) / 58194
Moving File...
From...
G:\C of A's\#CofA with background for web\1221TK@CK-4002.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\1221TK@CK-4002.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\1232TK@CT-0743.pdf
To...
G:\C of A's\#CofA with background for web\Missing CofAs\1232TK@CT-0743.pdf
Moving File...
From...
G:\C of A's\#CofA with background for web\1242TK@CE-4066.pdf
To...
G:\C of A's\#CofA with background for web\Missing CofAs\1242TK@CE-4066.pdf
Moving File...
From...
G:\C of A's\#CofA with background for web\1242TK@CG-2205.pdf
To...
G:\C of A's\#CofA with background for web\Missing CofAs\1242TK@CG-2205.pdf
Moving File...
From...
G:\C of A's\#CofA with background for web\1242TK@CP-1465.PDF
To...
G:\C

From...
G:\C of A's\#CofA with background for web\ALD-8315@CR-1183.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\ALD-8315@CR-1183.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\ALD-8315@CR-2738.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\ALD-8315@CR-2738.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\ALD-8315@CR-3827.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\ALD-8315@CR-3827.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\ALD-8315@CR-5559.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\ALD-8315@CR-5559.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\ALD-8315@CS-1394.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\ALD-8315@CS-1394.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\ALD-8315@CS-4646.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\ALD-8315@CS-4646.PDF
M

To...
G:\C of A's\#CofA with background for web\Missing CofAs\AM-170@CK-4308.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\AM-170@CM-0283.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\AM-170@CM-0283.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\AM-170@CM-0936.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\AM-170@CM-0936.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\AM-170@CM-1540.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\AM-170@CM-1540.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\AM-170@CM-1993.pdf
To...
G:\C of A's\#CofA with background for web\Missing CofAs\AM-170@CM-1993.pdf
Moving File...
From...
G:\C of A's\#CofA with background for web\AM-170@CM-2808.pdf
To...
G:\C of A's\#CofA with background for web\Missing CofAs\AM-170@CM-2808.pdf
Moving File...
From...
G:\C of A's\#CofA with background for web\AM-170@CM-3445.pdf
To...
G:\C

To...
G:\C of A's\#CofA with background for web\Missing CofAs\AM-180@CS-0113.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\AM-181@CR-0369.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\AM-181@CR-0369.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\AM-182@CR-0372.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\AM-182@CR-0372.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\AM-200@CC-2283A.pdf
To...
G:\C of A's\#CofA with background for web\Missing CofAs\AM-200@CC-2283A.pdf
Moving File...
From...
G:\C of A's\#CofA with background for web\AM-200@CC-2283Z.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\AM-200@CC-2283Z.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\AM-200@CM-1290.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\AM-200@CM-1290.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\AM-200@CM-1290A.PDF
To...

To...
G:\C of A's\#CofA with background for web\Missing CofAs\AMM-622@CJ-1077Z.pdf
Moving File...
From...
G:\C of A's\#CofA with background for web\AMM-622@CM-3219.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\AMM-622@CM-3219.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\AMM-622@CM-3219Z.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\AMM-622@CM-3219Z.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\AMM-802@CF-3955.pdf
To...
G:\C of A's\#CofA with background for web\Missing CofAs\AMM-802@CF-3955.pdf
Moving File...
From...
G:\C of A's\#CofA with background for web\AMM-802@CH-2204.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\AMM-802@CH-2204.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\AMM-802@CL-2226.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\AMM-802@CL-2226.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\AMM-802@CL-2226

...

To...
G:\C of A's\#CofA with background for web\Missing CofAs\US-134@CG-2713.pdf
Moving File...
From...
G:\C of A's\#CofA with background for web\US-134@CJ-1310.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\US-134@CJ-1310.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\US-134@CJ-3839.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\US-134@CJ-3839.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\US-134@CJ-3839A.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\US-134@CJ-3839A.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\US-134@CJ-3839Z.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\US-134@CJ-3839Z.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\US-134@CR-1458.pdf
To...
G:\C of A's\#CofA with background for web\Missing CofAs\US-134@CR-1458.pdf
Moving File...
From...
G:\C of A's\#CofA with background for web\US-135@CF-1465.PDF
To...


To...
G:\C of A's\#CofA with background for web\Missing CofAs\US-450@CR-3901.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\US-450@CS-3950.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\US-450@CS-3950.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\US-455@CS-0306.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\US-455@CS-0306.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\US-700002387@CK-4348.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\US-700002387@CK-4348.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\US-700002387@CK-4348A.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\US-700002387@CK-4348A.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\US-700002387@CK-4348Z.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\US-700002387@CK-4348Z.PDF
Moving File...
From...
G:\C of A's\#CofA with backgrou

To...
G:\C of A's\#CofA with background for web\Missing CofAs\USPM-467D@CE-0343A.pdf
Moving File...
From...
G:\C of A's\#CofA with background for web\USPM-467D@CE-0343Z.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\USPM-467D@CE-0343Z.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\USPM-467D@CH-2345.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\USPM-467D@CH-2345.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\USPM-467D@CH-2345Z.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\USPM-467D@CH-2345Z.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\USPM-467D@CM-5386.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\USPM-467D@CM-5386.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\USPM-467D@CM-5386A.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\USPM-467D@CM-5386A.PDF
Moving File...
From...
G:\C of A's\#CofA with backgrou

To...
G:\C of A's\#CofA with background for web\Missing CofAs\UST-110@CP-1684.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\UST-110@CP-5901.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\UST-110@CP-5901.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\UST-110@CT-0639.pdf
To...
G:\C of A's\#CofA with background for web\Missing CofAs\UST-110@CT-0639.pdf
Moving File...
From...
G:\C of A's\#CofA with background for web\UST-111@CE-1922A.pdf
To...
G:\C of A's\#CofA with background for web\Missing CofAs\UST-111@CE-1922A.pdf
Moving File...
From...
G:\C of A's\#CofA with background for web\UST-111@CE-1922Z.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\UST-111@CE-1922Z.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\UST-111@CK-0349Z.PDF
To...
G:\C of A's\#CofA with background for web\Missing CofAs\UST-111@CK-0349Z.PDF
Moving File...
From...
G:\C of A's\#CofA with background for web\UST-111@CP-2

To...
G:\C of A's\#CofA with background for web\Missing CofAs\WRK-103I@NT059059.pdf
Moving File...
From...
G:\C of A's\#CofA with background for web\WRK-103I@NT060702.pdf
To...
G:\C of A's\#CofA with background for web\Missing CofAs\WRK-103I@NT060702.pdf
Moving File...
From...
G:\C of A's\#CofA with background for web\WRK-103J@NT059060.pdf
To...
G:\C of A's\#CofA with background for web\Missing CofAs\WRK-103J@NT059060.pdf
Moving File...
From...
G:\C of A's\#CofA with background for web\WRK-103J@NT060713.pdf
To...
G:\C of A's\#CofA with background for web\Missing CofAs\WRK-103J@NT060713.pdf
Moving File...
From...
G:\C of A's\#CofA with background for web\WRK-103K@NT059061.pdf
To...
G:\C of A's\#CofA with background for web\Missing CofAs\WRK-103K@NT059061.pdf
Moving File...
From...
G:\C of A's\#CofA with background for web\WRK-103K@NT060714.pdf
To...
G:\C of A's\#CofA with background for web\Missing CofAs\WRK-103K@NT060714.pdf
Moving File...
From...
G:\C of A's\#CofA with background for 